# 查看数据

In [8]:
import pandas as pd
df1 = pd.read_excel('../data/春节档-评分_240227_1709000662.xlsx')
df2 = pd.read_excel('../data/春节档-评论_240227_1709000672.xlsx')

In [2]:
display(df1.head())
display(df2.head())

,nodeId,bizId,name,scorePersonCount,totalScore,scoreAvg,scoreDistribution,hottestComments,爬取时间
0,3635280,1454966,周处除三害,5,40,8.0,"{'2': 1, '8': 1, '10': 3}",[],2024-02-27 09:53:19
1,3635280,1451163,怒潮,16,122,7.6,"{'2': 4, '6': 1, '8': 1, '10': 10}",['这个片子打一百个热辣滚烫'],2024-02-27 09:53:19
2,3635280,1397890,缉恶,62,280,4.5,"{'2': 36, '4': 6, '6': 3, '8': 2, '10': 15}",['打戏挺好看'],2024-02-27 09:53:19
3,3635280,1395583,碟中谍4,72,480,6.7,"{'2': 20, '4': 6, '6': 10, '8': 2, '10': 34}",['爬迪拜塔的？天呐过去这么久了。。。。'],2024-02-27 09:53:19
4,3635280,1378232,照明商店,156,994,6.4,"{'2': 48, '4': 12, '6': 23, '8': 9, '10': 64}",['必须5分啊'],2024-02-27 09:53:19


,bizId,name,commentId,commentUserId,commentUserName,commentContent,lightCount,blackCount,score,publishTime,爬取时间
0,1451163,怒潮,1338073748,38630036,hupu_7ec582da0d377388,尤其是最后推理的过程，太惊艳了,0,0,10,2024-02-25 20:40:53,2024-02-27 10:05:10
1,1451163,怒潮,1340705787,105882683,虎扑JR1893942946,满分，尤其是后面直接绝了,0,0,10,2024-02-27 01:03:34,2024-02-27 10:05:10
2,1451163,怒潮,1340555677,101090677,虎扑JR0275176204,这个片子打一百个热辣滚烫,0,0,10,2024-02-27 09:00:52,2024-02-27 10:05:10
3,1397890,缉恶,1328277331,108375843,虎扑JR0561461161,咋没人,0,0,4,2024-02-21 01:56:32,2024-02-27 10:05:09
4,1397890,缉恶,1330525884,107956052,虎扑JR0092582909,没听过,0,0,0,2024-02-22 15:24:31,2024-02-27 10:05:09


In [4]:
df1['数据类型'] = '电影分数'
df1.to_csv('../data/score.csv', index=False)
df2['数据类型'] = '电影评论'
df2.to_csv('../data/comment.csv', index=False)

# 加载知识库，并向量化

In [9]:
from langchain_load_llm import LLM
from langchain_module import Chain, EmbeddingVectorDB, DocsLoader, TextSpliter, Retriever
from dotenv import load_dotenv
import warnings


# 加载 api
load_dotenv('../../key.env')
warnings.filterwarnings('ignore')

# llm
llm = LLM().zhipu_glm_4()

# embedding 模型
embedding_mode_path = '../../model/bge-small-zh-v1.5'
embedding_model = EmbeddingVectorDB.load_local_embedding_model(embedding_mode_path)

# 本地知识库加载，csv 的加载进来默认每一行是一个块，可以理解为分割过了
local_knowledge_score = '../data/score.csv'
local_knowledge_comment = '../data/comment.csv'
docs_score = DocsLoader.csv_loader(local_knowledge_score)
docs_comment = DocsLoader.csv_loader(local_knowledge_comment)
docs = docs_score + docs_comment

# 知识库向量化保存
movie_vector_path = '../data/movie_vector'
db = EmbeddingVectorDB.faiss_vector_db(docs, movie_vector_path, embedding_model)

创建向量数据库路径 =》 ../data/movie_vector


# 查询

## 周处除三害`这部电影的的平均评分是多少？

In [28]:
query = '`周处除三害`这部电影的的平均评分是多少？'
retriever = Retriever.similarity(db, query, topk=100)
retriever

[Document(page_content="nodeId: 3635280\nbizId: 1454966\nname: 周处除三害\nscorePersonCount: 5\ntotalScore: 40\nscoreAvg: 8.0\nscoreDistribution: {'2': 1, '8': 1, '10': 3}\nhottestComments: []\n爬取时间: 2024-02-27 09:53:19\n数据类型: 电影分数", metadata={'source': '../data/score.csv', 'row': 0}),
 Document(page_content='bizId: 1288602\nname: 我们一起摇太阳\ncommentId: 1316134937\ncommentUserId: 98057665\ncommentUserName: hupu_55b6f8afbc822ebf\ncommentContent: 所谓三部曲只是说这导演拍了三部关于绝症的电影，各自是毫无关联的，不用看\nlightCount: 1\nblackCount: 0\nscore: 0\npublishTime: 2024-02-14 06:20:19\n爬取时间: 2024-02-27 10:04:29\n数据类型: 电影评论', metadata={'source': '../data/comment.csv', 'row': 253}),
 Document(page_content='bizId: 1288400\nname: 热辣滚烫\ncommentId: 1324521980\ncommentUserId: 30006388\ncommentUserName: 詹黑变绿便黑詹\ncommentContent: 如果给贾玲评分那毫无疑问的五星，不管出于什么目的，减肥的毅力值得肯定。但这个电影是真的不行，从人物到背景都没有百元之恋的那种混乱社会下的无力与主角深陷泥泞挣扎的感觉，爆发的点也是莫名其妙的强硬，乐莹她并不是非拳击不可。更恐怖的是最后那段，那句“我已经赢了”让我浑身鸡皮疙瘩，并不是震撼，而是震惊，诚然现在社会高压下，很多人也会选择用阿Q的“精神胜利法”来降低胜利的标准，以求缓解压力，但这不应该被表达出来，更不应该通过

In [29]:
prompt = '''
参考以下信息回答问题：
```
{context}
```
--------------------------------
问题：{query}
'''
res = Chain.base_llm_chain(llm, prompt, query=query, context=retriever)
res

'根据文档内容，电影`周处除三害`的平均评分为8.0。'

## 春节期间上映了多少部电影？ 

In [62]:
query = '春节期间上映了多少部电影？电影名称的字段为`name`'
langchian_retriever = Retriever.similarity(db, query, topk=len(docs))
langchian_retriever

[Document(page_content='bizId: 1288601\nname: 第二十条\ncommentId: 1335067967\ncommentUserId: 101929087\ncommentUserName: 星河坠江海\ncommentContent: 感觉是春节最有意义的一档电影\nlightCount: 0\nblackCount: 0\nscore: 10\npublishTime: 2024-02-24 15:55:38\n爬取时间: 2024-02-27 10:03:28\n数据类型: 电影评论', metadata={'source': '../data/comment.csv', 'row': 715}),
 Document(page_content='bizId: 1374842\nname: 首尔之春\ncommentId: 1327030251\ncommentUserId: 97956283\ncommentUserName: Zhang氏家族发言人\ncommentContent: 春节看过最好的电影！[你再骂]\nlightCount: 0\nblackCount: 0\nscore: 10\npublishTime: 2024-02-20 02:36:58\n爬取时间: 2024-02-27 10:05:03\n数据类型: 电影评论', metadata={'source': '../data/comment.csv', 'row': 28}),
 Document(page_content='bizId: 1288399\nname: 飞驰人生2\ncommentId: 1325400495\ncommentUserId: 99320135\ncommentUserName: 春风秋色\ncommentContent: 春节档你最看的电影！\nlightCount: 0\nblackCount: 0\nscore: 10\npublishTime: 2024-02-19 00:05:28\n爬取时间: 2024-02-27 10:00:38\n数据类型: 电影评论', metadata={'source': '../data/comment.csv', 'row': 2007}),
 Document(pa

In [56]:
# 使用 llmaindex 后处理过滤
from llamaindex_module import RetriverRank


retriever = RetriverRank.langchain_retriver_convert_NodeWithScore(langchian_retriever, query)
display(len(retriever))

retriever = RetriverRank.keyword_node_postprocessor(retriever,  required_keywords=['电影分数'])
len(retriever)

5677

17

In [63]:
prompt = '''
参考以下信息回答问题：
```
{context}
```
--------------------------------
问题：{query}
'''
res = Chain.base_llm_chain(llm, prompt, query=query, context=retriever)
res

'根据提供的信息，春节期间上映了多部电影，具体电影名称如下：\n\n1. 周处除三害\n2. 怒潮\n3. 缉恶\n4. 碟中谍4\n5. 照明商店\n6. 红毯先生\n7. 喜羊羊与灰太狼之开心闯龙年\n8. 热辣滚烫\n9. 八戒之天蓬下界\n10. 首尔之春\n11. 第二十条\n12. 黄貔：天降财神猫\n13. 熊出没之逆转时空\n14. 目中无人2\n15. 一闪一闪亮晶晶\n16. 飞驰人生2\n17. 我们一起摇太阳\n\n共有17部电影在春节期间上映。'

#  怒潮这部电影的评论主要讲了什么？

In [71]:
query = '`怒潮`这部电影的评论主要讲了什么？'
langchian_retriever = Retriever.similarity_score_threshold(db, query, topk=100, score_threshold=0.3)
langchian_retriever

[Document(page_content='bizId: 1288400\nname: 热辣滚烫\ncommentId: 1336227629\ncommentUserId: 108479261\ncommentUserName: 虎扑JR0413539707\ncommentContent: 我非常喜欢热辣滚烫这部电影，大部分人把这部电影理解为是一部励志热血片，而我认为这部电影揭示了认知方法和思维模式对人命运的影响。电影开始的女主躺平，不想通过工作获取生存资源，当烂好人，不想通过提升身份地位和个人能力而是想用舍利和不拒绝别人来换取别人对自己的好。其核心的思维模式是回避现实和期望破格获取。后来女主在经历一系列打击背刺并跳楼无果之后开始转变自己的思维方式:尊重客观事实，尊重人性的本来面目，爱自己。许多人在遭遇伤痛，被人伤害之后都会变得愤世嫉俗，心理扭曲，痛恨伤害自己的每个人，这正是用情绪来回避客观事实的表现。而片中女主女主选择了接受尊重人性本来面目的客观事实，并倾注心血于自我完成和自我成长，并没有打击报复和怨恨伤害过她的人，没有做出“被毒蛇咬了之后追着蛇咬”之类伤害自己的行为，做到了真正的对自己好，为自己活，爱自己，真正赢了一次。\n文学、艺术就像药方子，对我的症不一定对别人的症，大家各有各的体会，多交流，多发现不同的角度。\nlightCount: 0\nblackCount: 0\nscore: 10\npublishTime: 2024-02-24 23:48:10\n爬取时间: 2024-02-27 09:55:44\n数据类型: 电影评论', metadata={'source': '../data/comment.csv', 'row': 5468}),
 Document(page_content='bizId: 1288400\nname: 热辣滚烫\ncommentId: 1327004284\ncommentUserId: 103076004\ncommentUserName: 虎扑JR1298273933\ncommentContent: 我想要客观的电影评论，虽然我也不看好这部翻拍日本电影的电影\nlightCount: 0\nblackCount: 0\nscore: 2\npublishTime: 2024-02-20 00:49:35

In [72]:
prompt = '''
参考以下信息回答问题：
```
{context}
```
--------------------------------
问题：{query}
'''
res = Chain.base_llm_chain(llm, prompt, query=query, context=langchian_retriever)
res

'根据提供的电影评论，关于《怒潮》的主要评论如下：\n\n1. 一位用户表示，这部电影非常棒，尤其是最后的推理部分令人惊艳。\n\n2. 另一位用户认为，《怒潮》这部电影可以打败100部《热辣滚烫》。\n\n3. 在电影评分数据中，怒潮的评分为7.6分，其中10分评价有10个，2分评价有4个。\n\n综上所述，《怒潮》这部电影受到一定的好评，尤其是结局部分的推理部分获得了较多用户的称赞。'